In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from app.database import SessionLocal, Base, engine
from app.models import *
from app import models
from app.utils import hash_password, reset_db
import requests
from sqlalchemy import func

import random
import uuid
from typing import List, Dict, Set, Tuple

import numpy as np
from faker import Faker
from app.models import (
    User,
    Group,
    GroupMembership,
    Report,
    Contest,
    ContestParticipation,
    Role,
)

In [3]:
def getContest(cid):
    url = 'https://codeforces.com/api/contest.standings'
    params = {
        'contestId': cid
    }
    res = requests.get(url, params).json()
    if res['status'] != 'OK':
        return None
    res = res['result']
    return {
        'contest': res['contest'],
        'standings': [
            {
                'handle': i['party']['members'][0]['handle'],
                'rank': i['rank'],
                'points': i['points'],
                'penalty': i['penalty']
            } for i in res['rows']
        ]
    }

cfhandles=set()
contests = [2050,2051,2109]
for contest in contests:
    standings = getContest(contest)['standings']
    for i in range(min(len(standings), 2000)):
        cfhandles.add(standings[i]['handle'])


In [4]:
cfhandles = list(cfhandles)
len(cfhandles)

5452

In [5]:
def random_subset(n, k):
    return random.sample([i for i in range(n)], k)

def pick_random_subset(population: list, k: int) -> list:
    """k distinct elements from population (k may be 0)."""
    if k == 0:
        return []
    return random.sample(population, k)

def get_mock_contest_standings(cid, include_users=None):
    if include_users is None:
        include_users = []
    res = {
        'contest': {
            'id': cid,
            'name': f'Codeforces Round {cid}',
            'type': 'CF',
            'phase': 'FINISHED',
            'frozen': False,
            'durationSeconds': 7200,
            'startTimeSeconds': 1746110100,
            'relativeTimeSeconds': 267545
        },
        'standings': [
        ]
    }
    num_participants = 8000
    standings = [
        {'handle': 'random-user'+str(i+1), 'rank': (i+1), 'points': 9754 - i, 'penalty': 0}
        for i in range(num_participants)
    ]
    res['standings'] = standings
    udx = pick_random_subset([i for i in range(num_participants)], len(include_users))

    for i in range(len(udx)):
        res['standings'][udx[i]]['handle'] = include_users[i]
    
    return res


In [6]:
"""
    simulate what happens at the actual database level when:
    
        - actual users register
        - a group is created by a user
        - a group moderator invites someone to join a group
        - a user requests to join a group
        - a group moderator accepts a pending join request
        - a user accepts a group join invite
        - a user is kicked out of a group
        - a user leaves a group

        - cron job pulls an upcoming contest from codeforces (or any other platform) - contests will be available for registration AFTER it gets pulled and added to db
        - a user registers for a contest within a group
        - a user revokes his contest registration within a group (can only be performed if time.now() < contest.start_date)
        - cron job pulls contest standings from cf and initiates rating recalc per group


        - a user creates a report object
        - a moderator/admin resolves a report object
        - ratings rollback for successful reports?
        
        - an announcement is created by siteadmins
"""

'\n    simulate what happens at the actual database level when:\n\n        - actual users register\n        - a group is created by a user\n        - a group moderator invites someone to join a group\n        - a user requests to join a group\n        - a group moderator accepts a pending join request\n        - a user accepts a group join invite\n        - a user is kicked out of a group\n        - a user leaves a group\n\n        - cron job pulls an upcoming contest from codeforces (or any other platform) - contests will be available for registration AFTER it gets pulled and added to db\n        - a user registers for a contest within a group\n        - a user revokes his contest registration within a group (can only be performed if time.now() < contest.start_date)\n        - cron job pulls contest standings from cf and initiates rating recalc per group\n\n\n        - a user creates a report object\n        - a moderator/admin resolves a report object\n        - ratings rollback for 

In [7]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
Faker.seed(SEED)
faker = Faker()

NUM_USERS = 5_000
NUM_GROUPS = 30
NUM_CONTESTS = 5
NUM_PARTICIPATIONS = 15_000
NUM_REPORTS = 100
NUM_ANNOUNCEMENTS = 30

DEFAULT_PASS = "devpass"

In [8]:
reset_db()
Base.metadata.create_all(bind=engine)

db = SessionLocal()

dropping all tables...
all tables dropped.
creating tables from models...
schema rebuilt.


In [9]:
# POPULATE USERS
# create admins and common group
admin_users = [
    User(
        user_id="shrey",
        role=Role.admin,
        cf_handle="negative-xp",
        email_id="dwxghcqcd@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
    User(
        user_id="ani",
        role=Role.admin,
        cf_handle="roomTemperatureIQ",
        email_id="chjd7639383@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
]

users = admin_users

for handle in cfhandles:
    uid = 'uid' + handle
    users.append(
        User(
            user_id=uid,
            role=Role.user,
            cf_handle=handle,
            atcoder_handle=None if random.random() < 0.5 else uid + "_ac",
            codechef_handle=None if random.random() < 0.7 else uid + "_cc",
            twitter_handle=None if random.random() < 0.6 else uid + "_tw",
            trusted_score=random.randint(0, 100),
            email_id=str(uid) + '@gmail.com',
            hashed_password=hash_password(DEFAULT_PASS),
        )
    )

# create 5000 users
while len(users) < NUM_USERS:
    uid = 'testUser' + str(len(users)-1)
    users.append(
        User(
            user_id=uid,
            role=Role.user,
            cf_handle=uid + '_cf',
            atcoder_handle=None if random.random() < 0.5 else uid + "_ac",
            codechef_handle=None if random.random() < 0.7 else uid + "_cc",
            twitter_handle=None if random.random() < 0.6 else uid + "_tw",
            trusted_score=random.randint(0, 100),
            email_id=str(uid) + '@gmail.com',
            hashed_password=hash_password(DEFAULT_PASS),
        )
    )

db.add_all(users)
db.commit()

user_df = pd.read_sql("SELECT * FROM users", engine)
print(user_df.shape)
user_df.head()

(5454, 11)


,user_id,role,cf_handle,atcoder_handle,codechef_handle,twitter_handle,internal_default_rated,trusted_score,email_id,hashed_password,timestamp
0,shrey,admin,negative-xp,None,None,None,True,88,dwxghcqcd@gmail.com,HASpEU8Z/yutWMVMYbf0DkZA0aUv3cgPqtDlLKcB2l1xpj...,2025-05-19 21:01:51.021233
1,ani,admin,roomTemperatureIQ,None,None,None,True,88,chjd7639383@gmail.com,sHlKY+cVZRBgx1mNw+hZrjlmIcTofgiThBR9btsjjag9c8...,2025-05-19 21:01:51.021233
2,uidnot_insync,user,not_insync,uidnot_insync_ac,None,None,True,28,uidnot_insync@gmail.com,LDeUUccXAJ1AepzRrevia669NPrX3RgyLCh9I5PVyGKonK...,2025-05-19 21:01:51.021233
3,uidpanospask,user,panospask,None,None,uidpanospask_tw,True,69,uidpanospask@gmail.com,uhC9e/PfJyYVp5llZl+1tmx/uXCEYUz0z0Z6meI+pj0EhX...,2025-05-19 21:01:51.021233
4,uidSatorouGojo,user,SatorouGojo,None,None,None,True,27,uidSatorouGojo@gmail.com,ZW+C3E5eWf3bzIqqhGDkdvCcAUstFZY0EZ2op4/amETKVo...,2025-05-19 21:01:51.021233


In [10]:
# POPULATE GROUPS

common_group = Group(
    group_id="main",
    group_name="main",
    group_description="group consisting of ALL users",
    is_private=False,
)

groups = [common_group]
for g_idx in range(1, NUM_GROUPS):
    g_id = f"g{g_idx:02d}"
    g_name = faker.unique.catch_phrase().lower().replace(" ", "‑")
    is_priv = random.random() < 0.3
    group = Group(
        group_id=g_id,
        group_name=g_name,
        group_description=faker.sentence(nb_words=10),
        is_private=is_priv,
    )
    groups.append(group)

db.add_all(groups)
db.commit()

group_df = pd.read_sql("SELECT * FROM groups", engine)
print(group_df.shape)
group_df.head()

(30, 5)


,group_id,group_name,group_description,is_private,timestamp
0,main,main,group consisting of ALL users,False,2025-05-19 21:01:54.784398
1,g01,sharable‑bifurcated‑algorithm,Each cause bill scientist nation opportunity a...,False,2025-05-19 21:01:54.784398
2,g02,robust‑4thgeneration‑open‑architecture,Respond red information last everything thank ...,False,2025-05-19 21:01:54.784398
3,g03,optimized‑global‑focus‑group,Democratic shake bill here grow gas enough.,False,2025-05-19 21:01:54.784398
4,g04,balanced‑upward-trending‑knowledgebase,By two bad fall pick those gun court attorney ...,True,2025-05-19 21:01:54.784398


In [13]:
groups[0].timestamp

datetime.datetime(2025, 5, 19, 21, 1, 54, 784398)

In [12]:
# POPULATE MEMBERSHIPS
memberships = []


# common group should have ALL users
memberships.append(GroupMembership(
    user_id=users[0].user_id,
    group_id=groups[0].group_id,
    role=Role.admin,
    user_group_rating=1888,
    user_group_max_rating=1888,
))

for i in range(1, len(users)):
    mxr = int(random.random() * 3000)
    cur = mxr - int(random.random() * (mxr/2))
    memberships.append(
        GroupMembership(
            user_id=users[i].user_id,
            group_id=groups[0].group_id,
            role=Role.user,
            user_group_rating=cur,
            user_group_max_rating=mxr,
        )
    )

# add memberships to other groups
size_palette = [5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597]
while len(size_palette) < len(groups) - 1:
    size_palette.append(random.randint(10, 800))
random.shuffle(size_palette)

user_idx = [i for i in range(len(users))]

for i in range(1, len(groups)):
    grp_size = size_palette[i-1]
    members = pick_random_subset(user_idx, grp_size)
    # admin for this group
    memberships.append(
        GroupMembership(
            user_id = users[members[0]].user_id,
            group_id = groups[i].group_id,
            role=Role.admin,
            user_group_rating=1500,
            user_group_max_rating=1500, 
        )
    )
    for j in members[1:]:
        memberships.append(
            GroupMembership(
                user_id=users[j].user_id,
                group_id=groups[i].group_id,
                role=Role.user,
                user_group_rating=1500,
                user_group_max_rating=1500,
            )
        )

db.add_all(memberships)
db.commit()

membership_df = pd.read_sql("SELECT * FROM group_memberships", engine)
print(membership_df.shape)
membership_df.head()

(15910, 7)


,user_id,group_id,role,user_group_rating,user_group_max_rating,status,timestamp
0,shrey,main,admin,1888,1888,active,2025-05-19 21:01:57.548756
1,ani,main,user,1055,1860,active,2025-05-19 21:01:57.548756
2,uidnot_insync,main,user,1483,1733,active,2025-05-19 21:01:57.548756
3,uidpanospask,main,user,1920,2565,active,2025-05-19 21:01:57.548756
4,uidSatorouGojo,main,user,658,661,active,2025-05-19 21:01:57.548756


In [14]:
membership_df = pd.read_sql("SELECT * FROM group_memberships", engine)
print(membership_df.shape)
membership_df.head()

(15910, 7)


,user_id,group_id,role,user_group_rating,user_group_max_rating,status,timestamp
0,shrey,main,admin,1888,1888,active,2025-05-19 21:01:57.548756
1,ani,main,user,1055,1860,active,2025-05-19 21:01:57.548756
2,uidnot_insync,main,user,1483,1733,active,2025-05-19 21:01:57.548756
3,uidpanospask,main,user,1920,2565,active,2025-05-19 21:01:57.548756
4,uidSatorouGojo,main,user,658,661,active,2025-05-19 21:01:57.548756


In [15]:
df = pd.read_sql("SELECT * FROM groups", engine)
df.head()

,group_id,group_name,group_description,is_private,timestamp
0,main,main,group consisting of ALL users,False,2025-05-19 21:01:54.784398
1,g01,sharable‑bifurcated‑algorithm,Each cause bill scientist nation opportunity a...,False,2025-05-19 21:01:54.784398
2,g02,robust‑4thgeneration‑open‑architecture,Respond red information last everything thank ...,False,2025-05-19 21:01:54.784398
3,g03,optimized‑global‑focus‑group,Democratic shake bill here grow gas enough.,False,2025-05-19 21:01:54.784398
4,g04,balanced‑upward-trending‑knowledgebase,By two bad fall pick those gun court attorney ...,True,2025-05-19 21:01:54.784398


In [16]:
# populate CONTESTS]

NUM_CONTESTS = 30
contests = []

actual = [2050,2051,2109]

for contest in actual:
    contests.append(
        Contest(
            contest_id = f"c{contest}",
            contest_name = f'Codeforces Contest {contest}',
            platform = "Codeforces",
            start_time_posix = int(random.random() * 1e9),
            duration_seconds = 7200,
            link = f'https://codeforces.com/contest/{contest}',
            finished=True,
            internal_contest_identifier = contest,
        )
    )


for i in range(NUM_CONTESTS):
    contests.append(
        Contest(
            contest_id = f"c{3000+i}",
            contest_name = f'Codeforces Contest {int(random.random() * 99999)}',
            platform = "Codeforces",
            start_time_posix = int(random.random() * 1e8),
            duration_seconds = 1938473,
            link = 'OCD',
            finished=False,
            internal_contest_identifier = f'{random.random() * 99999}',
        )
    )

db.add_all(contests)
db.commit()

contest_df = pd.read_sql("SELECT * FROM contests", engine)
print(contest_df.shape)
contest_df.head()

(33, 10)


,contest_id,contest_name,platform,start_time_posix,duration_seconds,link,internal_contest_identifier,standings,finished,timestamp
0,c2050,Codeforces Contest 2050,Codeforces,498017735,7200,https://codeforces.com/contest/2050,2050,None,True,2025-05-19 21:02:37.222011
1,c2051,Codeforces Contest 2051,Codeforces,96900856,7200,https://codeforces.com/contest/2051,2051,None,True,2025-05-19 21:02:37.222011
2,c2109,Codeforces Contest 2109,Codeforces,840143453,7200,https://codeforces.com/contest/2109,2109,None,True,2025-05-19 21:02:37.222011
3,c3000,Codeforces Contest 91475,Codeforces,63599929,1938473,OCD,28750.499342013307,None,False,2025-05-19 21:02:37.222011
4,c3001,Codeforces Contest 11726,Codeforces,5592703,1938473,OCD,5089.31527391485,None,False,2025-05-19 21:02:37.222011


In [17]:
# add finished contest participations to simulate delta
participations = []

for group in groups:
    members = group.memberships
    n_members = len(members)
    parts = random_subset(n_members, int(n_members//2))
    for ii in parts:

        contest = contests[0]
        be4 = members[ii].user_group_rating
        cur = be4 + ((-1)**(int(random.random() * 2))) * int(random.random()*80)
        
        participations.append(
            ContestParticipation(
                user_id = members[ii].user_id,
                group_id = group.group_id,
                contest_id = contest.contest_id,
                rank=int(random.random() * 2000),
                rating_before = be4,
                rating_after = cur,
            )
        )

        contest = contests[1]
        be4 = cur
        cur = be4 + ((-1)**(int(random.random() * 2))) * int(random.random()*80)
        
        participations.append(
            ContestParticipation(
                user_id = members[ii].user_id,
                group_id = group.group_id,
                contest_id = contest.contest_id,
                rank=int(random.random() * 2000),
                rating_before = be4,
                rating_after = cur,
            )
        )


        contest = contests[2]
        be4 = cur
        cur = be4 + ((-1)**(int(random.random() * 2))) * int(random.random()*80)
        
        participations.append(
            ContestParticipation(
                user_id = members[ii].user_id,
                group_id = group.group_id,
                contest_id = contest.contest_id,
                rank=int(random.random() * 2000),
                rating_before = be4,
                rating_after = cur,
            )
        )

db.add_all(participations)
db.commit()

participation_df = pd.read_sql("SELECT * FROM contest_participations", engine)
print(participation_df.shape)
participation_df.head()

(23841, 7)


,user_id,group_id,contest_id,rank,rating_before,rating_after,timestamp
0,uiddilip_patnana,main,c2050,1719,1588,1555,2025-05-19 21:04:09.014979
1,uiddilip_patnana,main,c2051,1778,1555,1589,2025-05-19 21:04:09.014979
2,uiddilip_patnana,main,c2109,318,1589,1527,2025-05-19 21:04:09.014979
3,uidbrendonw,main,c2050,777,530,465,2025-05-19 21:04:09.014979
4,uidbrendonw,main,c2051,1152,465,460,2025-05-19 21:04:09.014979


In [20]:
participation_df = pd.read_sql("SELECT * FROM contest_participations", engine)
print(participation_df.shape)
participation_df.head()

(23841, 7)


,user_id,group_id,contest_id,rank,rating_before,rating_after,timestamp
0,uiddilip_patnana,main,c2050,1719,1588,1555,2025-05-19 21:04:09.014979
1,uiddilip_patnana,main,c2051,1778,1555,1589,2025-05-19 21:04:09.014979
2,uiddilip_patnana,main,c2109,318,1589,1527,2025-05-19 21:04:09.014979
3,uidbrendonw,main,c2050,777,530,465,2025-05-19 21:04:09.014979
4,uidbrendonw,main,c2051,1152,465,460,2025-05-19 21:04:09.014979


In [21]:
db.rollback()

In [22]:
# populate contest participations
# unfinished_participations = []
# for i in range(3, len(contests)):
#     for group in groups:
#         members = group.memberships
#         n_members = len(members)
#         parts = random_subset(n_members, int(n_members//10))
#         for ii in parts:
#             unfinished_participations.append(
#                 ContestParticipation(
#                     user_id = members[ii].user_id,
#                     group_id = group.group_id,
#                     contest_id = contest.contest_id,
#                     rating_before = members[ii].user_group_rating,
#                 )
#             )

# db.add_all(unfinished_participations)
# db.commit()

# participation_df = pd.read_sql("SELECT * FROM contest_participations", engine)
# print(participation_df.shape)
# participation_df.head()

In [23]:
# populate reports
n_parts = len(participations)
reports = []

while len(reports) < NUM_REPORTS:
    idx = int(random.random() * n_parts)
    rp = participations[idx]
    
    grp = db.query(Group).filter(Group.group_id == rp.group_id).all()[0]
    members = grp.memberships
    reporter = members[int(len(members) * random.random())]
    reports.append(
        Report(
            report_id = f"report{len(reports)}",
            group_id = rp.group_id,
            contest_id = rp.contest_id,
            reporter_user_id = reporter.user_id,
            respondent_user_id = rp.user_id,
            report_description = faker.sentence(nb_words=12)
        )
    )

db.add_all(reports)
db.commit()

report_df = pd.read_sql("SELECT * FROM reports", engine)
print(report_df.shape)
report_df.head()

(100, 11)


,report_id,group_id,contest_id,reporter_user_id,respondent_user_id,report_description,resolved,resolved_by,resolve_message,resolve_timestamp,timestamp
0,report0,main,c2051,uidNotChatGPT,uidMazza,Voice boy wife condition while enter board its...,False,None,None,None,2025-05-19 21:08:16.083614
1,report1,main,c2051,uidYashaswiR,uidaltgifted,Tonight couple and job mind southern rather vo...,False,None,None,None,2025-05-19 21:08:16.083614
2,report2,g25,c2050,uidApurv_M,uidkzyKT,Finish summer rest feel finally impact I fast ...,False,None,None,None,2025-05-19 21:08:16.083614
3,report3,main,c2109,uidmr360_,uidHeeluu,Fight decision size parent focus kid put.,False,None,None,None,2025-05-19 21:08:16.083614
4,report4,g03,c2051,uidLuoxuanLove,uidmsab3f,List top somebody college be middle plan frien...,False,None,None,None,2025-05-19 21:08:16.083614


In [24]:
# populate announcememts

NUM_ANNOUNCEMENTS = 40
announcements = []
for i in range(NUM_ANNOUNCEMENTS):
    announcements.append(
        Announcement(
            announcement_id = f"anmt{i}",
            group_id = "main",
            title = faker.sentence(nb_words=7),
            content = faker.sentence(nb_words=100)
        )
    )

db.add_all(announcements)
db.commit()

announcement_df = pd.read_sql("SELECT * FROM announcements", engine)
print(announcement_df.shape)
announcement_df.head()

(40, 5)


,announcement_id,group_id,title,content,timestamp
0,anmt0,main,Consider whom item treat area buy check clearl...,Generation wait thus suffer economy play nearl...,2025-05-19 21:08:23.872789
1,anmt1,main,No guy eye hit late near stay perhaps particul...,Window hour some fund voice sense current meet...,2025-05-19 21:08:23.872789
2,anmt2,main,Machine whatever everything fear walk word sid...,First give value somebody event business quali...,2025-05-19 21:08:23.872789
3,anmt3,main,Reduce tree serious soon stay seven quite.,Their bank land region back nor article natura...,2025-05-19 21:08:23.872789
4,anmt4,main,Help painting always authority source onto.,Describe decade trade field training deep coup...,2025-05-19 21:08:23.872789


In [16]:
db.rollback()

In [17]:
pd.read_sql("SELECT * from contests", engine).head()

,contest_id,contest_name,platform,start_time_posix,duration_seconds,link,internal_contest_identifier,standings,finished,timestamp
0,c3000,Codeforces Contest 63832,Codeforces,55026232,1938473,OCD,86881.63745479863,None,True,2025-05-18 14:16:55.467864
1,c3001,Codeforces Contest 89374,Codeforces,96166673,1938473,OCD,79206.65716526027,None,False,2025-05-18 14:16:55.467864
2,c3002,Codeforces Contest 24471,Codeforces,4311207,1938473,OCD,6505.914582732978,None,False,2025-05-18 14:16:55.467864
3,c3003,Codeforces Contest 52936,Codeforces,3316581,1938473,OCD,48006.27120483181,None,False,2025-05-18 14:16:55.467864
4,c3004,Codeforces Contest 61354,Codeforces,58193444,1938473,OCD,1766.1345674021998,None,True,2025-05-18 14:16:55.467864


In [18]:
# update ratings for a contest across ALL groups

# TODO

In [19]:
pd.read_sql("SELECT * from users", engine).head()

,user_id,role,cf_handle,atcoder_handle,codechef_handle,twitter_handle,internal_default_rated,trusted_score,email_id,hashed_password,timestamp
0,shrey,admin,negative-xp,None,None,None,True,88,dwxghcqcd@gmail.com,cuytEO2Ej970KzgBNZr688dfA1hA6yag61GKpqKZ257eww...,2025-05-18 14:16:54.004751
1,ani,admin,roomTemperatureIQ,None,None,None,True,88,chjd7639383@gmail.com,IbjrF7OGOiq3K5XX3gQdY+xBYs9rrw9/F1tYtvK5eAyXTz...,2025-05-18 14:16:54.004751
2,testUser1,user,testUser1_cf,testUser1_ac,None,None,True,28,testUser1@gmail.com,kteDpvqWe3R7XAZ4zRelazN2zuIX96tfE1h4ACApf0FoBA...,2025-05-18 14:16:54.004751
3,testUser2,user,testUser2_cf,None,None,testUser2_tw,True,69,testUser2@gmail.com,OfDfmzMOi+4+GA+4E6eBG2H6/+UBmGaRtdcLHD6Xv47bIo...,2025-05-18 14:16:54.004751
4,testUser3,user,testUser3_cf,None,None,None,True,27,testUser3@gmail.com,3Ss06ndsyMTeCXUaU1hsQOESGky3dmi/dNNbnJpLb8WiS5...,2025-05-18 14:16:54.004751


In [20]:
from sqlalchemy import func
from sqlalchemy.orm import Session
from typing import List, Tuple
from app import models

def get_group_member_counts(db: Session) -> List[Tuple[str, int]]:
    """
    Returns a list of (group_id, member_count) for all groups.
    Groups with 0 members are included.
    """
    results = (
        db.query(
            models.Group.group_id,
            func.count(models.GroupMembership.user_id).label("member_count")
        )
        .outerjoin(models.GroupMembership, models.Group.group_id == models.GroupMembership.group_id)
        .group_by(models.Group.group_id)
        .all()
    )
    return results


get_group_member_counts(db)


[('g10', 391),
 ('g01', 233),
 ('main', 5000),
 ('g20', 21),
 ('g05', 165),
 ('g06', 726),
 ('g23', 673),
 ('g29', 13),
 ('g04', 89),
 ('g09', 759),
 ('g24', 345),
 ('g02', 610),
 ('g28', 8),
 ('g14', 78),
 ('g15', 621),
 ('g03', 377),
 ('g13', 695),
 ('g11', 1597),
 ('g22', 463),
 ('g19', 310),
 ('g18', 398),
 ('g16', 95),
 ('g07', 144),
 ('g08', 34),
 ('g12', 55),
 ('g21', 432),
 ('g17', 987),
 ('g27', 64),
 ('g25', 5),
 ('g26', 473)]

In [21]:
db.rollback()

In [22]:
res = (
        db.query(
            models.Group,
            func.count(models.GroupMembership.user_id).label("member_count")
        )
        .outerjoin(models.GroupMembership, models.Group.group_id == models.GroupMembership.group_id)
        .group_by(models.Group.group_id)
        .all()
    )

In [23]:
res

[(<Group(id=g10, name='mandatory‑bifurcated‑frame')>, 391),
 (<Group(id=g01, name='sharable‑bifurcated‑algorithm')>, 233),
 (<Group(id=main, name='main')>, 5000),
 (<Group(id=g20, name='configurable‑contextually-based‑architecture')>, 21),
 (<Group(id=g05, name='focused‑bandwidth-monitored‑implementation')>, 165),
 (<Group(id=g06, name='public-key‑upward-trending‑encryption')>, 726),
 (<Group(id=g23, name='innovative‑stable‑benchmark')>, 673),
 (<Group(id=g29, name='focused‑directional‑algorithm')>, 13),
 (<Group(id=g04, name='balanced‑upward-trending‑knowledgebase')>, 89),
 (<Group(id=g09, name='right-sized‑asymmetric‑info-mediaries')>, 759),
 (<Group(id=g24, name='synergized‑scalable‑firmware')>, 345),
 (<Group(id=g02, name='robust‑4thgeneration‑open‑architecture')>, 610),
 (<Group(id=g28, name='user-centric‑dynamic‑flexibility')>, 8),
 (<Group(id=g14, name='face-to-face‑asymmetric‑graphical‑user‑interface')>, 78),
 (<Group(id=g15, name='automated‑full-range‑archive')>, 621),
 (<Grou

In [24]:
res[0][0].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x108d6ce30>,
 'group_name': 'mandatory‑bifurcated‑frame',
 'is_private': False,
 'group_id': 'g10',
 'group_description': 'Citizen about reveal rest will seven medical blood personal success medical.',
 'timestamp': datetime.datetime(2025, 5, 18, 14, 16, 54, 141802)}

In [25]:
res[0][1]

391

In [26]:
pd.read_sql("SELECT * from users", engine).head()

,user_id,role,cf_handle,atcoder_handle,codechef_handle,twitter_handle,internal_default_rated,trusted_score,email_id,hashed_password,timestamp
0,shrey,admin,negative-xp,None,None,None,True,88,dwxghcqcd@gmail.com,cuytEO2Ej970KzgBNZr688dfA1hA6yag61GKpqKZ257eww...,2025-05-18 14:16:54.004751
1,ani,admin,roomTemperatureIQ,None,None,None,True,88,chjd7639383@gmail.com,IbjrF7OGOiq3K5XX3gQdY+xBYs9rrw9/F1tYtvK5eAyXTz...,2025-05-18 14:16:54.004751
2,testUser1,user,testUser1_cf,testUser1_ac,None,None,True,28,testUser1@gmail.com,kteDpvqWe3R7XAZ4zRelazN2zuIX96tfE1h4ACApf0FoBA...,2025-05-18 14:16:54.004751
3,testUser2,user,testUser2_cf,None,None,testUser2_tw,True,69,testUser2@gmail.com,OfDfmzMOi+4+GA+4E6eBG2H6/+UBmGaRtdcLHD6Xv47bIo...,2025-05-18 14:16:54.004751
4,testUser3,user,testUser3_cf,None,None,None,True,27,testUser3@gmail.com,3Ss06ndsyMTeCXUaU1hsQOESGky3dmi/dNNbnJpLb8WiS5...,2025-05-18 14:16:54.004751
